In [45]:
import pandas as pd

rcp = pd.read_csv("C:/project/company/yk_cmp_rpt.csv")

In [140]:
rcp = ['20120419000097']

In [46]:
fin = pd.concat(concat_list, sort=False , axis =0)
fin.to_excel("C:/project/vv/a652.xlsx")

27674

In [106]:
from html_table_parser import parser_functions as parser
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re

# 재무제표 정규표현식 #
fin_stat_code1 = re.compile("현[ \s]*금[ \s]*및[ \s]*((예[ \s]*치[ \s]*금)|(현[ \s]*금[ \s]*((성[ \s]*자[ \s]*산)|(등[ \s]*가[ \s]*물))))")
fin_stat_code2 = re.compile("자[ \s]*본[ \s]*총[ \s]*계")
fin_stat_code3 = re.compile("자[ \s]*산[ \s]*총[ \s]*계")
 
# 재무제표 금액단위 #
re_unit1 = re.compile('단[\s]*위[ \s]*:[ \s]*원') # 원
re_unit2 = re.compile('단[\s]*위[ \s]*:[ \s]*천[\s]*') # 천원
re_unit3 = re.compile('단[\s]*위[ \s]*:[ \s]*백[\s]*만[\s]*') # 백만원
re_unit4 = re.compile('단[\s]*위[ \s]*:[ \s]*C[ \s]*N[ \s]*Y[ \s]*') # 위안
re_unit5 = re.compile('단[\s]*위[ \s]*:[ \s]*R[ \s]*M[ \s]*B[ \s]*') # 위안
re_unit6 = re.compile('단[\s]*위[ \s]*:[ \s]*C[ \s]*N[ \s]*H[ \s]*') # 위안
re_unit7 = re.compile('단[\s]*위[ \s]*:[ \s]*J[ \s]*P[ \s]*Y[ \s]*') # 엔
re_unit8 = re.compile('단[\s]*위[ \s]*:[ \s]*U[ \s]*S[ \s]*D[ \s]*') # 달러
re_unit9 = re.compile('단[\s]*위[ \s]*:[ \s]*억[ \s]*원[\s]*')


concat_list = []
#보고서 번호 입력 #
for idx, rep_num in enumerate(rcp['rcp_no'][114867:]):
#for idx, rep_num in enumerate(rcp):
    #if idx % 70 ==0:
    print(idx, '번째 보고서 번호는 :', rep_num , '입니다 .')
              
    # URL, CONTENT #
    rep_url      = """http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}""".format(rep_num)
    rep_page     = requests.get(rep_url)
    rep_menu     = BeautifulSoup(rep_page.content,'html.parser')
    rep_menu_txt = rep_menu.text

    
    # 재무제표 위치값 찾기 #
    key1 = rep_menu_txt.find("2. 연결재무제표")    ; key2 = rep_menu_txt.find("4. 연결재무제표")
    key3 = rep_menu_txt.find("4. 재무제표")        ; key4 = rep_menu_txt.find("재무제표 등")
    key5 = rep_menu_txt.find("재무에 관한")        ; key6 = rep_menu_txt.find('3. 재무제표')
    key7 = rep_menu_txt.find('연 결 재 무 제 표')  ; key8 = rep_menu_txt.find('반 기 재 무 제 표')
    
    key_basket = [ key1, key2, key3, key4, key5, key6, key7, key8 ]
    key_basket = list(filter(lambda x: x != -1 , key_basket))
    
    # 키워드가 없어서 다음 보고서로 넘어감 #
    if len(key_basket) == 0 :
        continue
        
    for key in key_basket:
        x = rep_menu_txt[key:key+170] ; x = x[:x.find('d\'')+2]
        
        # URL 데이터 추출
        raw_dcm = x.split(',')[4] ; raw_ele = x.split(',')[5] ; raw_off = x.split(',')[6]
        raw_leg = x.split(',')[7] ; raw_dtd = x.split(',')[8].replace("\'","")
        
        dcm = raw_dcm.replace("\'","").strip()
        ele = raw_ele.replace("\'","").strip()
        off = raw_off.replace("\'","").strip()
        leg = raw_leg.replace("\'","").strip()
        dtd = raw_dtd.replace("\'","").strip()

        fin_stat_url = """http://dart.fss.or.kr/report/viewer.do"""
        fin_stat_par = {'rcpNo':rep_num ,'dcmNo': dcm,'eleId': ele,'offset': off,'length': leg,'dtd': dtd}

        fin_stat_page = requests.get(fin_stat_url, fin_stat_par)
        fin_stat_rslt = BeautifulSoup(fin_stat_page.content,'html.parser')

        check_table = fin_stat_rslt.find_all('table')
        check_unit =  fin_stat_rslt.find_all('body')[0].text[:700]

#----------------------------------------------------------------------------------------------------------------------------- 
        
        if len(check_table) < 2:
            continue
        
        table_loc = 0
        cnt       = 0
        for table in check_table:
            if fin_stat_code1.search(table.text):
                table_loc = cnt
                break
            elif fin_stat_code2.search(table.text):
                table_loc = cnt
                break
            elif fin_stat_code3.search(table.text):
                table_loc = cnt
                break
            cnt = cnt + 1
        
        balance_table = check_table[table_loc]
        unit_table = check_table[table_loc-1]
        
#-------단위금액--------------------------------------------------------------------------------------------------------------     
        
        unit = 0
        if len(unit_table(string=re_unit1)) != 0:
            unit = 1
        elif len(unit_table(string=re_unit2)) != 0:
            unit = 2
        elif len(unit_table(string=re_unit3)) != 0:
            unit = 3
        elif len(unit_table(string=re_unit4)) != 0:
            unit = 4
        elif len(unit_table(string=re_unit5)) != 0:
            unit = 5
        elif len(unit_table(string=re_unit6)) != 0:
            unit = 6
        elif len(unit_table(string=re_unit7)) != 0:
            unit = 7
        elif len(unit_table(string=re_unit8)) != 0:
            unit = 8
        elif len(unit_table(string=re_unit9)) != 0:
            unit = 9
            
        if unit ==0:
            if re_unit1.search(check_unit):
                unit = 1
            elif re_unit2.search(check_unit):
                unit = 2
            elif re_unit3.search(check_unit):
                unit = 3
            elif re_unit4.search(check_unit):
                unit = 4
            elif re_unit5.search(check_unit):
                unit = 5
            elif re_unit6.search(check_unit):
                unit = 6
            elif re_unit7.search(check_unit):
                unit = 7
            elif re_unit8.search(check_unit):
                unit = 8
            elif re_unit9.search(check_unit):
                unit = 9

#-----------------------------------------------------------------------------------------------------------------------------
        table_2d    = parser.make2d(balance_table)
        fin_stat_df = pd.DataFrame(table_2d)
        fin_stat_df = fin_stat_df.fillna('')
        
        if len(fin_stat_df.columns) < 2 or len(fin_stat_df[0]) < 14:
            continue
            
        fin_stat_df.replace(regex=['[\s]+','[.]+\.'], value='', inplace=True)
        
        if len(fin_stat_df.isin(['주석'])[1].value_counts()) == 2:
            fin_stat_df.drop(1, axis=1, inplace=True)
            
        
        fin_stat_df.drop_duplicates(0,inplace=True, keep='first')
        fin_stat_df.reset_index(drop=True,inplace=True)
        
        fin_stat_df[0].replace(regex=['\(.+\)'], value='', inplace=True)
        fin_stat_df.iloc[:2,:] = fin_stat_df.iloc[:2,:].replace(regex=['\(.+\)'], value='')
        fin_stat_df.iloc[2:,1:] = fin_stat_df.iloc[2:,1:].replace(regex=['[()]'], value='')
        
        fin_stat_df.iloc[0,0] = '과목'
        fin_stat_df[0].replace(regex=['[^가-힣]'],value='',inplace=True)
        
        dump_name = []
        for idx, col in enumerate(fin_stat_df[0]):
            if len(col) >= 9 or len(col)==0:
                dump_name.append(idx)
                
        fin_stat_df.drop(dump_name, 0, inplace=True)
        fin_stat_df.reset_index(drop=True,inplace=True)
        
        #fin_stat_df[0].replace(regex=['[^가-힣]'],value='',inplace=True)

        if len(fin_stat_df.columns) == 2:
            
            dump_col = []
            for idx, col in enumerate(fin_stat_df.iloc[:,1]):
                if len(col) >= 19 :
                    dump_col.append(idx)

            fin_stat_df.drop(dump_col, 0, inplace=True)
            fin_stat_df.reset_index(drop=True,inplace=True)
            
            fin_stat_df.iloc[1:,1:].replace(regex=['△', 'Δ'], value = '-', inplace=True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['\(.+\)'], value = '', inplace =True)
            fin_stat_df.iloc[1:,1:].replace(regex=['[^-0-9]'], value = '', inplace =True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['-$'], value = '', inplace =True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['--+'], value = '-', inplace =True)
            fin_stat_df.iloc[1:,1:].replace(to_replace=['','-','0'], value = np.nan,inplace=True)

            result_df = fin_stat_df
            result_df.dropna(inplace=True)
            
        
        elif fin_stat_df.iloc[0,1] == fin_stat_df.iloc[0,2]:
            try:
                dump_col1 = []
                for idx, col in enumerate(fin_stat_df.iloc[:,1]):
                    if len(col) >= 19 :
                        dump_col.append(idx)

                fin_stat_df.drop(dump_col1, 0, inplace=True)
                fin_stat_df.reset_index(drop=True,inplace=True)


                dump_col2 = []
                for idx, col in enumerate(fin_stat_df.iloc[:,2]):
                    if len(col) >= 19 :
                        dump_col.append(idx)

                fin_stat_df.drop(dump_col2, 0, inplace=True)
                fin_stat_df.reset_index(drop=True,inplace=True)
                #try:

                fin_stat_df = fin_stat_df.iloc[:,:3]

                fin_stat_df.iloc[1:,1:].replace(regex=['△', 'Δ'], value = '-', inplace =True)
                #fin_stat_df.iloc[1:,1:].replace(regex=['\(.+\)'], value = '', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(regex=['[^-0-9]'], value = '', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(regex=['--+'], value = '-', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(regex=['-$'], value = '', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(to_replace=['','-','0'], value = 0,inplace=True)

                fin_stat_df.iloc[1:,1:] = fin_stat_df.iloc[1:,1:].astype('float64')


                left = fin_stat_df[0] ; fin_stat_df.iloc[0,2] = ''
                right = fin_stat_df.iloc[:,1] + fin_stat_df.iloc[:,2]

                result_df = pd.DataFrame({0:left, 1:right})
                result_df.replace(to_replace=[0],value=np.nan, inplace=True)
                result_df.dropna(inplace=True)
            except:
                continue
        
        elif fin_stat_df.iloc[0,2] == '':
            try:
                dump_col1 = []
                for idx, col in enumerate(fin_stat_df.iloc[:,1]):
                    if len(col) >= 19 :
                        dump_col.append(idx)

                fin_stat_df.drop(dump_col1, 0, inplace=True)
                fin_stat_df.reset_index(drop=True,inplace=True)


                dump_col2 = []
                for idx, col in enumerate(fin_stat_df.iloc[:,2]):
                    if len(col) >= 19 :
                        dump_col.append(idx)

                fin_stat_df.drop(dump_col2, 0, inplace=True)
                fin_stat_df.reset_index(drop=True,inplace=True)


                fin_stat_df = fin_stat_df.iloc[:,:3]

                fin_stat_df.iloc[1:,1:].replace(regex=['△', 'Δ'], value = '-', inplace =True)
                #fin_stat_df.iloc[1:,1:].replace(regex=['\(.+\)'], value = '', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(regex=['[^-0-9]'], value = '', inplace =True)
                #fin_stat_df.iloc[1:,1:].replace(regex=['--+'], value = '-', inplace =True)
                #fin_stat_df.iloc[1:,1:].replace(regex=['-$'], value = '', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(to_replace=['','-','0'], value = 0,inplace=True)

                fin_stat_df.iloc[1:,1:] = fin_stat_df.iloc[1:,1:].astype('int64')


                left = fin_stat_df[0] ; fin_stat_df.iloc[0,2] = ''
                right = fin_stat_df.iloc[:,1] + fin_stat_df.iloc[:,2]

                result_df = pd.DataFrame({0:left, 1:right})
                result_df.replace(to_replace=[0],value=np.nan, inplace=True)
                result_df.dropna(inplace=True)
            except:
                continue
            
        else:
            fin_stat_df = fin_stat_df.iloc[:,:2]

            fin_stat_df.iloc[1:,1:].replace(regex=['△', 'Δ'], value = '-', inplace=True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['\(.+\)'], value = '', inplace =True)
            fin_stat_df.iloc[1:,1:].replace(regex=['[^-0-9]'], value = '', inplace =True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['--+'], value = '-', inplace =True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['-$'], value = '', inplace =True)
            fin_stat_df.iloc[1:,1:].replace(to_replace=['','-','0'], value = np.nan,inplace=True)
            result_df = fin_stat_df
            result_df.dropna(inplace=True)
            
        
        
        result_df.drop_duplicates(0, inplace=True, keep='first')
        result_df.reset_index(drop=True, inplace=True)
        result_df = result_df.set_index(0).T
        
        # 데이터 추가 #
      
        if len(result_df.columns) > 11:
            result_df['rcp_no'] = rep_num
            result_df['unit'] = unit
            concat_list.append(result_df)
            break
        
        else:
            continue
            

            
fin = pd.concat(concat_list, sort=False , axis =0)
fin.to_excel("C:/project/vv/ae.xlsx")

            
        


        

0 번째 보고서 번호는 : 20130329001601 입니다 .
1 번째 보고서 번호는 : 20121114000116 입니다 .
2 번째 보고서 번호는 : 20120814000227 입니다 .
3 번째 보고서 번호는 : 20120515001181 입니다 .
4 번째 보고서 번호는 : 20120330000531 입니다 .
5 번째 보고서 번호는 : 20111114000639 입니다 .
6 번째 보고서 번호는 : 20110816000628 입니다 .
7 번째 보고서 번호는 : 20110516001371 입니다 .
8 번째 보고서 번호는 : 20110331001145 입니다 .
9 번째 보고서 번호는 : 20101115000100 입니다 .
10 번째 보고서 번호는 : 20100813000764 입니다 .
11 번째 보고서 번호는 : 20100517000731 입니다 .
12 번째 보고서 번호는 : 20100331001084 입니다 .
13 번째 보고서 번호는 : 20091113000512 입니다 .
14 번째 보고서 번호는 : 20090814000502 입니다 .
15 번째 보고서 번호는 : 20090515000783 입니다 .
16 번째 보고서 번호는 : 20090331000296 입니다 .
17 번째 보고서 번호는 : 20081114000157 입니다 .
18 번째 보고서 번호는 : 20080814000349 입니다 .
19 번째 보고서 번호는 : 20080515000385 입니다 .
20 번째 보고서 번호는 : 20080509000863 입니다 .
21 번째 보고서 번호는 : 20080331001422 입니다 .
22 번째 보고서 번호는 : 20071114000819 입니다 .
23 번째 보고서 번호는 : 20070814000929 입니다 .
24 번째 보고서 번호는 : 20070515001183 입니다 .
25 번째 보고서 번호는 : 20070330000740 입니다 .
26 번째 보고서 번호는 : 20061114001077 입니다 .
27 번째 보고서 번

219 번째 보고서 번호는 : 20160513003443 입니다 .
220 번째 보고서 번호는 : 20160329000641 입니다 .
221 번째 보고서 번호는 : 20151113000637 입니다 .
222 번째 보고서 번호는 : 20150813000571 입니다 .
223 번째 보고서 번호는 : 20150514005046 입니다 .
224 번째 보고서 번호는 : 20150330001026 입니다 .
225 번째 보고서 번호는 : 20141126000213 입니다 .
226 번째 보고서 번호는 : 20140828000577 입니다 .
227 번째 보고서 번호는 : 20140523000223 입니다 .
228 번째 보고서 번호는 : 20140328001140 입니다 .
229 번째 보고서 번호는 : 20131125000340 입니다 .
230 번째 보고서 번호는 : 20130828000471 입니다 .
231 번째 보고서 번호는 : 20130515001089 입니다 .
232 번째 보고서 번호는 : 20130328000637 입니다 .
233 번째 보고서 번호는 : 20121114000043 입니다 .
234 번째 보고서 번호는 : 20120813000340 입니다 .
235 번째 보고서 번호는 : 20120514003409 입니다 .
236 번째 보고서 번호는 : 20120329000543 입니다 .
237 번째 보고서 번호는 : 20111110000173 입니다 .
238 번째 보고서 번호는 : 20110812000645 입니다 .
239 번째 보고서 번호는 : 20110513003392 입니다 .
240 번째 보고서 번호는 : 20110329000327 입니다 .
241 번째 보고서 번호는 : 20101108000166 입니다 .
242 번째 보고서 번호는 : 20100813000104 입니다 .
243 번째 보고서 번호는 : 20100513002352 입니다 .
244 번째 보고서 번호는 : 20100330000662 입니다 .
245 번째 보고서 번

435 번째 보고서 번호는 : 20031125000108 입니다 .
436 번째 보고서 번호는 : 20031113000008 입니다 .
437 번째 보고서 번호는 : 20030814000142 입니다 .
438 번째 보고서 번호는 : 20030515000015 입니다 .
439 번째 보고서 번호는 : 20030328000262 입니다 .
440 번째 보고서 번호는 : 20021114000083 입니다 .
441 번째 보고서 번호는 : 20020813000617 입니다 .
442 번째 보고서 번호는 : 20020524000095 입니다 .
443 번째 보고서 번호는 : 20020514000004 입니다 .
444 번째 보고서 번호는 : 20020401000117 입니다 .
445 번째 보고서 번호는 : 20011113000038 입니다 .
446 번째 보고서 번호는 : 20010813000004 입니다 .
447 번째 보고서 번호는 : 20010514000119 입니다 .
448 번째 보고서 번호는 : 20010329000031 입니다 .
449 번째 보고서 번호는 : 20001113000130 입니다 .
450 번째 보고서 번호는 : 20000811000217 입니다 .
451 번째 보고서 번호는 : 20000512000007 입니다 .
452 번째 보고서 번호는 : 20000328000074 입니다 .
453 번째 보고서 번호는 : 20180813000284 입니다 .
454 번째 보고서 번호는 : 20180514001949 입니다 .
455 번째 보고서 번호는 : 20180326000370 입니다 .
456 번째 보고서 번호는 : 20171113000288 입니다 .
457 번째 보고서 번호는 : 20170811000165 입니다 .
458 번째 보고서 번호는 : 20170512002784 입니다 .
459 번째 보고서 번호는 : 20170331000806 입니다 .
460 번째 보고서 번호는 : 20161114000257 입니다 .
461 번째 보고서 번

651 번째 보고서 번호는 : 20060313000046 입니다 .
652 번째 보고서 번호는 : 20051109000011 입니다 .
653 번째 보고서 번호는 : 20050811000366 입니다 .
654 번째 보고서 번호는 : 20050507000032 입니다 .
655 번째 보고서 번호는 : 20050428000166 입니다 .
656 번째 보고서 번호는 : 20050329000588 입니다 .
657 번째 보고서 번호는 : 20041112000246 입니다 .
658 번째 보고서 번호는 : 20040813000786 입니다 .
659 번째 보고서 번호는 : 20040622000144 입니다 .
660 번째 보고서 번호는 : 20040524000105 입니다 .
661 번째 보고서 번호는 : 20040514000673 입니다 .
662 번째 보고서 번호는 : 20040511000061 입니다 .
663 번째 보고서 번호는 : 20040330000201 입니다 .
664 번째 보고서 번호는 : 20031204000202 입니다 .
665 번째 보고서 번호는 : 20031201000028 입니다 .
666 번째 보고서 번호는 : 20031113000330 입니다 .
667 번째 보고서 번호는 : 20030813000283 입니다 .
668 번째 보고서 번호는 : 20030514000705 입니다 .
669 번째 보고서 번호는 : 20030415000138 입니다 .
670 번째 보고서 번호는 : 20030327000089 입니다 .
671 번째 보고서 번호는 : 20030327000016 입니다 .
672 번째 보고서 번호는 : 20021113000268 입니다 .
673 번째 보고서 번호는 : 20020813000016 입니다 .
674 번째 보고서 번호는 : 20020514000108 입니다 .
675 번째 보고서 번호는 : 20020327000258 입니다 .
676 번째 보고서 번호는 : 20011114000039 입니다 .
677 번째 보고서 번

867 번째 보고서 번호는 : 20101115000200 입니다 .
868 번째 보고서 번호는 : 20100813000294 입니다 .
869 번째 보고서 번호는 : 20100514001167 입니다 .
870 번째 보고서 번호는 : 20100331000103 입니다 .
871 번째 보고서 번호는 : 20091113000021 입니다 .
872 번째 보고서 번호는 : 20090814000649 입니다 .
873 번째 보고서 번호는 : 20090515000234 입니다 .
874 번째 보고서 번호는 : 20090331000159 입니다 .
875 번째 보고서 번호는 : 20081113000088 입니다 .
876 번째 보고서 번호는 : 20080814000756 입니다 .
877 번째 보고서 번호는 : 20080515001349 입니다 .
878 번째 보고서 번호는 : 20080331001859 입니다 .
879 번째 보고서 번호는 : 20071114000130 입니다 .
880 번째 보고서 번호는 : 20070813000382 입니다 .
881 번째 보고서 번호는 : 20070515000724 입니다 .
882 번째 보고서 번호는 : 20070330001478 입니다 .
883 번째 보고서 번호는 : 20061114000396 입니다 .
884 번째 보고서 번호는 : 20060814000471 입니다 .
885 번째 보고서 번호는 : 20060515001190 입니다 .
886 번째 보고서 번호는 : 20060515000536 입니다 .
887 번째 보고서 번호는 : 20060330001677 입니다 .
888 번째 보고서 번호는 : 20051114000092 입니다 .
889 번째 보고서 번호는 : 20050816000225 입니다 .
890 번째 보고서 번호는 : 20050513001047 입니다 .
891 번째 보고서 번호는 : 20050429000494 입니다 .
892 번째 보고서 번호는 : 20050331001002 입니다 .
893 번째 보고서 번

1081 번째 보고서 번호는 : 20000331000310 입니다 .
1082 번째 보고서 번호는 : 20180813000421 입니다 .
1083 번째 보고서 번호는 : 20180514004418 입니다 .
1084 번째 보고서 번호는 : 20180329000695 입니다 .
1085 번째 보고서 번호는 : 20171113000524 입니다 .
1086 번째 보고서 번호는 : 20170811000599 입니다 .
1087 번째 보고서 번호는 : 20170512004927 입니다 .
1088 번째 보고서 번호는 : 20170330000291 입니다 .
1089 번째 보고서 번호는 : 20161111000398 입니다 .
1090 번째 보고서 번호는 : 20160812000734 입니다 .
1091 번째 보고서 번호는 : 20160513004839 입니다 .
1092 번째 보고서 번호는 : 20160328000417 입니다 .
1093 번째 보고서 번호는 : 20151113000483 입니다 .
1094 번째 보고서 번호는 : 20150813000463 입니다 .
1095 번째 보고서 번호는 : 20150514000650 입니다 .
1096 번째 보고서 번호는 : 20150327000371 입니다 .
1097 번째 보고서 번호는 : 20141127000085 입니다 .
1098 번째 보고서 번호는 : 20140828000114 입니다 .
1099 번째 보고서 번호는 : 20140529000102 입니다 .
1100 번째 보고서 번호는 : 20140328000403 입니다 .
1101 번째 보고서 번호는 : 20131128000021 입니다 .
1102 번째 보고서 번호는 : 20130828000031 입니다 .
1103 번째 보고서 번호는 : 20130524000022 입니다 .
1104 번째 보고서 번호는 : 20130510000018 입니다 .
1105 번째 보고서 번호는 : 20130329000630 입니다 .
1106 번째 보고서 번호는 : 2012111